In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Task Details**

The prices for each index are quoted in terms of the national currency of the country each stock exchange is located in. By translating all of the indexes to a common currency, it would be easier to compare them.

The basic task is to translate all indexes to a common currency and the example is given with USD. 

For example, the Hong Kong Stock Exchange (HSI) is quoted in HKD, so our basic DataFrame will contain OHLC prices in HKD while our processed DataFrame will also include a translation to USD. 

There is no way to find the exchange rates from the basic DataFrame alone without pulling in external data in some form. This could be done easily enough by incorporating even a single point in time exchange rate and back-filling (as the author of the processed DataFrame has done). 

However, we may want to compare index prices in terms of a currency other than USD. This can easily be done if we know at least one other exchange rate, in this case their exchange rates against the USD.

We are not given the bid and ask, so we will assume the prices given are the midpoint (or one or the other, as long as it is consistent). 

For example, we can use the two quotes CAD/USD and USD/HKD to calculate the market-implied quote on the CAD/HKD cross rate. 

Begin by considering the transactions required to sell CAD and buy HKD, going through the CAD/USD and USD/HKD currency pairs. 

We can view this process intuitively as:

(Sell CAD, Buy HKD) = (Sell CAD, Buy USD) + (Sell USD, Buy HKD)

Note that the "Buy USD" and "Sell USD" in the expressions on the right-hand side of the equal sign will cancel out to give the CAD/HKD cross rate. 

In equation form, we can represent this relationship as follows: 

$$(\frac{CAD}{HKD}) = (\frac{CAD}{USD})(\frac{USD}{HKD})$$

Now, let's incorporate the bid-offer rates in order to do the CAD/HKD calculation. 

A rule of thumb is that when we speak of a bid or offer exchange rate, we are referring to the bid or offer for the currency in the denominator (the base currency). 

So, in the above equation, we are looking to "Buy HKD" in the denominator, so we are looking for an **offer** rate, thus we will be calculating the offer rate for CAD/HKD. 

Next, we look at the first term on the right side of the equation, we are going to "Buy USD", so again, we need the offer rate but this time for CAD/USD. 

Finally, when we look at the second term on the right side of the equation, we get "Buy HKD" in the denominator, so again, we need the offer rate but this time for USD/HKD. 

This is all to say that when we are calculating these rates, if we are looking to buy or sell, they simply need to all be the offer or bid rates. 

**CONVERSION**: Depending on the quotes provided, it may be necessary to invert one of the quotes in order to complete the calculation. 

Consider the case of calculating the implied GBP/EUR cross rate if you are given USD/GBP and USD/EUR quotes. 

Simply using the provided quotes will not generate the desired GBP/EUR cross rate. 

Instead, because the USD is in the numerator in both currency pairs, we will have to invert one of the pairs to derive the GBP/EUR cross rate. 

We can invert the USD/GBP quote and use that in our calculation instead. 

In [ ]:
# Load the DataFrames
df = pd.read_csv('../input/stock-exchange-data/indexData.csv')
df_p = pd.read_csv('../input/stock-exchange-data/indexProcessed.csv')
info = pd.read_csv('../input/stock-exchange-data/indexInfo.csv')

In [ ]:
# Basic Data
df.head()

In [ ]:
# Processed Data (w/ USD Close)
df_p.head()

In [ ]:
# Index information
info

Given prices in USD as well as the national currency of the index home country, we can find the corresponding rate with base USD:

In [ ]:
df_p['Exchange_Rate'] = df_p.Close / df_p.CloseUSD

In [ ]:
plt.plot(df_p[df_p.Index == 'HSI'].Exchange_Rate)

The exchange rate does not fluctuate. In other words, the ratio between the Close prices in the listed currency and the USD Close prices does not change. 

This indicates that the ratio used is a point in time ratio probably corresponding to the day the database was constructed rather than the more realistic ratio present at the date of the observation. In other words, the USD Close of the HSI on January of 2021 was likely based on a different exchange rate than the one used to find the USD Close in January of 2020 unless the rate has been pegged. In the processed DataFrame, the ratios are not realistic.

We can easily calculate all other exchange rates against the USD as follows:

In [ ]:
rates = {}
for x in info.Index.tolist():
    values = df_p[df_p.Index == x].Exchange_Rate.values
    name = '{}_USD'.format(info[info.Index == x].Currency.values[0])
    if name not in rates:
        if len(values) > 0:
            rates[name] = values[0]
            print('{0}:{1}'.format(name, float(values[0])))
        else:
            rates[name] = 'NaN'
            print('{0}:{1}'.format(name, 'NaN'))
    else:
        pass

We only get a 'NaN' on the Korea Exchange ('KS11') index, meaning the rates for it are not present in the processed DataFrame. It is, however, present in the basic DataFrame. 

It is the only index listed in KRW, and we know no other exchange rates with KRW so we cannot translate it.

In [ ]:
df_p[df_p.Index == 'KS11']

In [ ]:
df[df.Index == 'KS11'].head()

Given the different exchange rates with base USD, we can simply multiply the rate by the USD Close to find the price at a different base currency than either the USD or the national currency price provided.

In [ ]:
for k in rates:
    col = 'Close'+k[:3]
    if rates[k] == 'NaN':
        df_p[col] = 'NaN'
    else:
        df_p[col] = df_p.CloseUSD * rates[k]

In [ ]:
df_p.head()

Now, we may also want to find the different cross rates such as HKD_CNY, JPY_EUR, etc. and to do so we need to invert the price currency against base USD and multiply it by the new base currency exchange rate against base USD as follows: 

In [ ]:
# HKD_CNY exchange rate:
(1/rates['HKD_USD']) *  rates['CNY_USD']

In [ ]:
new_rates = {}
for k in rates:
    if (k[:3] == 'USD') | (rates[k] == 'NaN'):
        pass
    else:
        r = 1/rates[k]
        for j in rates:
            if (j[4:] == 'USD'):
                if (k[:3] != j[4:]) & (k[:3] != j[:3]):
                    name = '{0}_{1}'.format(k[:3], j[:3])
                    if name not in rates:
                        if np.isnan(float(rates[j])):
                            pass
                        else:
                            new_rates[name] = float(r) * float(rates[j])
                    else:
                        pass
                else:
                    pass
            else:
                pass

In [ ]:
new_rates

We now have a complete list of 83 exchange rates assuming no arbitrage conditions hold. 